## GPT V3

单个模型(fixed), 直接对接到webui (生成很多个), 所以产生一行一个的结果

In [1]:
from tqdm.auto import tqdm
import gradio as gr
import transformers
import torch
import os


import os
import hashlib
import json
from datetime import datetime
import concurrent.futures


# Assuming DEVICE is already defined
DEVICE = 'cuda'  # Use 'cuda' for GPU or 'cpu' for CPU

# Path to the models' directories
MODEL_DIRS = [
    'pixiv-prompts-gpt-finetunes/8xh100_run2_e2_s50k',
]

# Load the model and tokenizer from the directory
MODEL_NAME = os.path.basename(os.path.normpath(MODEL_DIRS[0]))
model = transformers.GPT2LMHeadModel.from_pretrained(MODEL_DIRS[0]).to(DEVICE).eval()
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_DIRS[0])

def generate_text(rating: str, date: str, quality: str, character: str, prompt: str, max_length: int, num_lines: int):
    """Generate text based on input prompt."""
    query_prompt = f'<input rating="{rating}" chara="{character}" date="{date}" quality="{quality}" tags="{prompt}">'
    query_prompt += "<output>"

    all_outputs = []
    for _ in range(num_lines):
        input_ids = tokenizer.encode(query_prompt, return_tensors='pt').to(DEVICE)
        output_sequences = model.generate(
            input_ids=input_ids,
            max_length=max_length + len(input_ids[0]),
            temperature=1.0,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.0,
            do_sample=True,
            num_return_sequences=1,
        )

        generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

        # Find and remove the initial part up to <output>
        start_output = generated_text.find("<output>") + len("<output>")
        generated_text = generated_text[start_output:].strip()

        # Remove the ending </output> tag or truncate at last complete tag
        end_tag = generated_text.find("</output>")
        if end_tag != -1:
            generated_text = generated_text[:end_tag]
        else:
            last_comma = generated_text.rfind(",")
            if last_comma != -1:
                generated_text = generated_text[:last_comma]

        all_outputs.append(generated_text.strip())

    # Join all outputs into a single string separated by new lines
    return "\n".join(all_outputs)

In [2]:
IMG_OUTPUT_DIR = "generated_images"


def save_images_and_metadata(gend_images, gend_text, prompt):
    # Ensure the output directory exists
    os.makedirs(IMG_OUTPUT_DIR, exist_ok=True)

    for image in gend_images:
        # Get filename
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
        img_hash = hashlib.md5(image.tobytes()).hexdigest()        
        file_name = f"{timestamp}_{img_hash[:4]}.webp"        
        file_path = os.path.join(IMG_OUTPUT_DIR, file_name)
        
        # Save image
        image.save(file_path, format='WEBP')

        # Create metadata JSON
        metadata = {
            "prompt": prompt,
            "generated_text": gend_text,
        }
        json_file_name = f"{timestamp}_{img_hash[:4]}.json"
        json_file_path = os.path.join(IMG_OUTPUT_DIR, json_file_name)
        
        with open(json_file_path, 'w') as json_file:
            json.dump(metadata, json_file)

In [3]:
import unibox as ub

from dataproc4.utils.string_utils import safe_split_tag_string, safe_concat


def preproc_idf_dict(idf_dict: dict) -> dict:
    """
    Convert underscores in keys and values of the idf_dict to spaces.
    Split each value string into a list, remove items that are keys in the idf_dict, 
    and then join the list back into a string.
    """
    new_idf_dict = {}
    for key, value in idf_dict.items():
        new_key = key.replace("_", " ")
        value_list = safe_split_tag_string(value.replace("_", " "))
        
        # Remove items that are keys in the idf_dict
        value_list = [item for item in value_list if item not in idf_dict]
        
        new_value = ", ".join(value_list)
        new_idf_dict[new_key] = new_value
    return new_idf_dict


idf_uri = "s3://bucket-data-catalog/data_catalog/silver_mini/dset_danbooru_character_related_tags_idf_thres3.5.json"
idf_dict = preproc_idf_dict(ub.loads(idf_uri))

chara_tags = "yoisaki kanade"
general_tags = ""


import random

def augment_prompt(chara_tags:str, general_tags:str, idf_dict:dict):
    
    # print(f"Received: chara tags: {chara_tags}, general tags: {general_tags}")

    # chara_list = safe_split_tag_string(chara_tags)
    # chara_list = [i.replace("_", " ") for i in chara_list] # convert to danbooru format

    # new_general_tags = safe_split_tag_string(general_tags)

    # for chara in chara_list:
    #     if chara in idf_dict:
    #         idf_candidates = safe_split_tag_string(idf_dict[chara])
    #         # random select up to 4 tags
    #         if len(idf_candidates) > 2:
    #             idf_candidates = random.sample(idf_candidates, 2)
    #         else:
    #             idf_candidates = idf_candidates

    #         new_general_tags += idf_candidates
    
    # new_general_tags = safe_concat(list(set(new_general_tags)))

    # print(f"Augmented general tags: {new_general_tags}")
    
    # return new_general_tags
    return general_tags


print(augment_prompt("yoisaki_kanade", "", idf_dict))
print(augment_prompt("ootori emu", "", idf_dict))

2024-07-17 07:04:57,102 [INFO] UniLogger: UniLoader.loads: .json LOADED from "/tmp/tmpohsd16ey/dset_danbooru_character_related_tags_idf_thres3.5.json" in 0.19s


In [4]:
def split_chara_and_gneral(tag_str:str):
    tags = safe_split_tag_string(tag_str)
    tags = [i.replace("_", " ") for i in tags]

    chara_tags = []
    general_tags = []
    for tag in tags:
        if tag in idf_dict:
            chara_tags.append(tag)
        else:
            general_tags.append(tag)
    return safe_concat(chara_tags), safe_concat(general_tags)

chara, general = split_chara_and_gneral("yoisaki kanade, ootori emu")

chara, general = split_chara_and_gneral("1girl, yellow eyes, selfie, hair over one eye, black pantyhose, gothic lolita, white hair, long hair")
print(chara)
print(general)


1girl, yellow eyes, selfie, hair over one eye, black pantyhose, gothic lolita, white hair, long hair


In [5]:
from typing import List, Tuple
from concurrent.futures import ThreadPoolExecutor
from run_comfy_api import run_workflow, DEFAULT_NEG

import os
HARDCODED_MODEL_NAME = os.path.basename(os.path.normpath(MODEL_DIRS[0]))


# LONG_DEFAULT_TAG_STR_END = "good quality, highres, 2020s, aesthetic, absurdres, [best quality]"

LONG_DEFAULT_TAG_STR_END = "2020s"

# LONG_DEFAULT_NEG = "nsfw, lowres, (bad), error, fewer, extra, missing, worst quality, jpeg artifacts, bad quality, watermark, unfinished, displeasing, chromatic aberration, signature, extra digits, artistic error, average, username, scan, [abstract]"

LONG_DEFAULT_NEG = DEFAULT_NEG + ", chromatic aberration, signature, extra digits, artistic error, scan, [abstract]"


# USE_MODEL = "bxl-v4c-stepfix/checkpoint-e11_s38000.safetensors"
USE_MODEL = "yada_checkpoints/qft_v4c_z3zz4_32_d16_sched/checkpoint-e83_s2000.safetensors"

def gen_text_and_gen_image(rating: str, date: str, quality: str, character: str, prompt: str, max_length: int, 
                           img_count: int, seed: int, tags_front: str, tags_back: str, augment_input: bool) -> Tuple[str, List[str]]:
    
    gend_text = []
    gend_images = []

    orig_input = f"Character: [{character}] | General: [{prompt}]"
    print(f"Get input: {orig_input}")

    with ThreadPoolExecutor() as executor:
        for _ in range(img_count):
            if augment_input:
                prompt = augment_prompt(character, prompt, idf_dict)
                text_future = executor.submit(generate_text, rating, date, quality, character, prompt, max_length, 1)
                txt = text_future.result().split("\n")[0]
                print(f"Generated text: {txt}")
            else:
                txt = safe_concat([character, prompt])
                print(f"Using original input: {txt}")

            # Add quality tags
            txt = safe_concat([tags_front, txt, tags_back])
            gend_text.append(txt)

            image_future = executor.submit(run_workflow, pos=txt, 
                                           model = USE_MODEL,
                                           neg=LONG_DEFAULT_NEG,
                                           seed=seed, batch_size=1)
            _gend_images = image_future.result()
            gend_images.extend(_gend_images)

    # Save images and metadata in the background
    with ThreadPoolExecutor() as executor:
        executor.submit(save_images_and_metadata, gend_images, gend_text, orig_input)
    
    return "\n\n".join(gend_text), gend_images
    

# Wrapper function for the default tab with preset parameters
def gen_text_and_gen_image_default(rating, prompt, num_images, augment_input):
    date = "2020s"
    quality = "excellent"
    
    chara, general = split_chara_and_gneral(prompt)
    text, img = gen_text_and_gen_image(rating, date, quality, chara, general, max_length= 220, 
                                  img_count=num_images, seed=-1, tags_front="good quality", tags_back="best quality, 2020s",
                                  augment_input=augment_input,
                                  )
    return img, text

# Define Gradio interface components for the advanced tab
import os
checkbox_choices = [os.path.basename(os.path.normpath(model_dir)) for model_dir in MODEL_DIRS]

advanced_inputs = [
    gr.Radio(choices=["general", "nsfw"], label="Rating", value="general"),
    gr.Radio(choices=["2000s", "2010s", "2020s"], label="Date", value="2020s"),
    gr.Radio(choices=["normal", "good", "excellent"], label="Quality", value="excellent"),
    gr.Textbox(lines=1, placeholder="hatsune miku", label="Character tags", autofocus=True),
    gr.Textbox(lines=2, placeholder="1girl, long hair, looking at viewer", label="General Tags", autofocus=True),
    gr.Slider(minimum=40, maximum=300, value=220, step=10, label="Max Prompt Length"),
    gr.Slider(minimum=1, maximum=4, value=2, step=1, label="Image Generation Count"),
    gr.Number(value=-1, label="Image Generation Seed"),
    gr.Textbox(lines=1, value="good quality", label="tags_front"),
    gr.Textbox(lines=1, value="best quality, 2020s", label="tags_back"),
    gr.Checkbox(label="Augment input", value=True),
]

advanced_outputs = [
    gr.Textbox(label="Generated Texts"),
    gr.Gallery(label="Generated Images", height=768),
]

# Define Gradio interface components for the default tab
default_inputs = [
    gr.Radio(choices=["general", "nsfw"], label="Rating", value="general"),
    # gr.Textbox(lines=1, placeholder="hatsune miku", label="Character tags"),
    gr.Textbox(lines=3, placeholder="hatsune miku, 1girl, short hair, eating", label="Character and General Tags  (Auto-detection supports *exact* danbooru tags for now)", autofocus=True),
    gr.Slider(minimum=1, maximum=8, value=2, step=1, label="Image Generation Count"),
    gr.Checkbox(label="Augment input", value=True),
]

default_outputs = [
    gr.Gallery(label="Generated Images", height=512),
    gr.Textbox(label="Generated Texts"),
]

# Define the interface with two tabs
iface = gr.TabbedInterface(
    [
        gr.Interface(
            fn=gen_text_and_gen_image_default,
            inputs=default_inputs,
            outputs=default_outputs,
            title=f"Default",
            description=f"Simple interface with preset values. (Shift+Enter to run) \n\nusing model: {USE_MODEL}",
        ),
        gr.Interface(
            fn=gen_text_and_gen_image,
            inputs=advanced_inputs,
            outputs=advanced_outputs,
            title=f"Advanced",
            description=f"Advanced interface with customizable parameters. (Shift+Enter to run)\n\nusing model: {USE_MODEL}",
        ),
    ],
    ["Default", "Advanced"],
    title="""

# Prompt Augment and SDXL New Model Demo
    
Pipeline:
  - input some danbooru tags (or danbooru-like tags)
  - augment the input to match the model training distribution
  - generate images from the augmented input (by calling ComfyUI API)
  - receives output

Input tags is in danbooru format (or similar): 
  - see: https://danbooru.donmai.us/posts/7792929

**Use The "Advanced" Tab for more granular controls**

"""
)

# Launch the interface
iface.launch(share=True, server_port=9876)

Running on local URL:  http://127.0.0.1:9876
Running on public URL: https://df9bb26c42e629e5d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Get input: Character: [] | General: []
Generated text: 1girl, animal ears, cat ears, barefoot, black hair, looking at viewer, animal ear fluff, cat girl, red eyes, strap slip, indoors, on couch, very long hair, knees up, red ribbon, bare shoulders, window, hair ribbon, sleeveless dress, white dress, bangs, eyebrows visible through hair, frilled dress, potted plant, bare arms, closed mouth, sunlight, armchair, tail raised, picture frame, collarbone, white flower, pillow, neck ribbon, holding animal, full body, crossed ankles, hair between eyes, hair intakes, hands on own knees, spaghetti strap, stuffed toy


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: multiple girls, swimsuit, 2girls, blue eyes, blonde hair, pink hair, red eyes, barefoot, jewelry, ponytail, holding hands, very long hair, bracelet, toenail polish, blue nails, thigh strap, looking at viewer, medium breasts, open mouth, interlocked fingers, cleavage, black bikini, standing, star (sky), outdoors, full body, night sky, halterneck, white bikini, star (symbol), bare legs, :d, criss-cross halter, bangs, hair ornament, small breasts, collarbone, side-tie bikini bottom, white ribbon, stomach, yuri, original works, bare shoulders, closed mouth, dutch angle, holding towel, leaning forward


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: yoisaki kanade, blindfold, 1girl, cum in pussy, nipples, bangs, solo, bdsm, choker, spread legs, open mouth, black collar, collar, long hair, lying, dress pull, cumdrip, flat chest, r-18, blindfolf, blue ribbon, clothes pull, cum overflow, crying, empty eyes, implied yuri, pillow, saliva, strapless dress, trembling, video camera, white blindfold, white theme


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Get input: Character: [] | General: [1girl]
Generated text: 1girl, red eyes, black hair, large breasts, white gloves, looking at viewer, cleavage, garter straps, black legwear, covered navel, peaked cap, skindentation, sitting, bare shoulders, black leotard, bangs, white headwear, indoors, curtains, arm up, very long hair, parted lips, strapless leotard, highleg leotard, on couch, collarbone, hand on headwear, window, arm behind head, black thighhighs, first-rate breasts, sexy clothes, ai illustration


Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.10/site-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
  File "/home/ubuntu/miniconda3/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/ubuntu/miniconda3/lib/python3.10/site-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/home/ubuntu/miniconda3/lib/python3.10/site-packages/gradio/blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/ubuntu/miniconda3/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/ubuntu/miniconda3/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/home/ubuntu/m